# Titanic - Machine Learning from Disaster

## Kaggle API install

In [1]:
# !pip install kaggle

In [2]:
# !kaggle competitions list

In [3]:
# !kaggle competitions download titanic

## Overview

[Start here!](https://www.kaggle.com/competitions/titanic/overview) Predict survival on the Titanic and get familiar with ML basics

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

In [4]:
import numpy as np
import pandas as pd

## Импорт данных

In [5]:
df = pd.read_csv('data/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Анализ данных

### Описание данных

| Variable | Definition                                  | Key                                            |
|----------|---------------------------------------------|------------------------------------------------|
| survival | Survival	                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                                | 	1 = 1st, 2 = 2nd, 3 = 3rd                     |
| sex	     | Sex                                         |                                                |	
| Age      | 	Age in years                               |                                                |	
| sibsp    | 	# of siblings / spouses aboard the Titanic |                                                |	
| parch    | 	# of parents / children aboard the Titanic |                                                |	
| ticket   | 	Ticket number                              |                                                |	
| fare     | 	Passenger fare                             |                                                |	
| cabin    | 	Cabin number                               |                                                |	
| embarked | 	Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes
**pclass**: A proxy for socio-economic status (SES)

1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...

**Sibling** = brother, sister, stepbrother, stepsister

**Spouse** = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...

**Parent** = mother, father

**Child** = daughter, son, stepdaughter, stepson

**Some** children travelled only with a nanny, therefore parch=0 for them.


In [6]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
PassengerId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Survived,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,1,0,1,0,1
Pclass,3,1,3,1,3,3,1,3,3,2,3,1,3,3,3,2,3,2,3,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry","Moran, Mr. James","McCarthy, Mr. Timothy J","Palsson, Master. Gosta Leonard","Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)","Nasser, Mrs. Nicholas (Adele Achem)","Sandstrom, Miss. Marguerite Rut","Bonnell, Miss. Elizabeth","Saundercock, Mr. William Henry","Andersson, Mr. Anders Johan","Vestrom, Miss. Hulda Amanda Adolfina","Hewlett, Mrs. (Mary D Kingcome)","Rice, Master. Eugene","Williams, Mr. Charles Eugene","Vander Planke, Mrs. Julius (Emelia Maria Vande...","Masselmani, Mrs. Fatima"
Sex,male,female,female,female,male,male,male,male,female,female,female,female,male,male,female,female,male,male,female,female
Age,22.0,38.0,26.0,35.0,35.0,NaN,54.0,2.0,27.0,14.0,4.0,58.0,20.0,39.0,14.0,55.0,2.0,NaN,31.0,NaN
SibSp,1,1,0,1,0,0,0,3,0,1,1,0,0,1,0,0,4,0,1,0
Parch,0,0,0,0,0,0,0,1,2,0,1,0,0,5,0,0,1,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450,330877,17463,349909,347742,237736,PP 9549,113783,A/5. 2151,347082,350406,248706,382652,244373,345763,2649
Fare,7.25,71.2833,7.925,53.1,8.05,8.4583,51.8625,21.075,11.1333,30.0708,16.7,26.55,8.05,31.275,7.8542,16.0,29.125,13.0,18.0,7.225


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
df.Survived.value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

> Дисбаланс невысокий

In [10]:
df.Pclass.value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [11]:
df.Age.notna().value_counts()

Age
True     714
False    177
Name: count, dtype: int64

In [12]:
df[df.Age.isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


### Возраст  - 177 пропусков

> Если возраст не указан м.б. младенец и есть корреляция с суммой  

In [13]:
df[((df.Fare == 0) & (df.Pclass == 3)) & df.Age.notna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
179,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,NaN,S
271,272,1,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,NaN,S
302,303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,NaN,S
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0,NaN,S


### Проверка на корреляцию

In [14]:
df[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']].corr(method='pearson')

,Pclass,Age,SibSp,Parch,Fare
Pclass,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,-0.549500,0.096067,0.159651,0.216225,1.000000


## Обработка данных

### Разделение на выборки 

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
target = df['Survived']
features = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)
X_train.shape, X_test.shape

((596, 7), (295, 7))

In [18]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,male,54.0,0,0,51.8625,S
718,3,male,NaN,0,0,15.5000,Q


## Создание конвейера

План:

1. Численные:
 - Age - заполнение пропусков
2. Категорийный:
 - Embarked, Sex - Кодирование

In [19]:
import pandas as pd
from sklearn.compose import ColumnTransformer

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier


### Итоговый конвейер

In [20]:
pipe_cat = Pipeline([('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan))])

pipe_num = Pipeline([('knn', KNNImputer(n_neighbors=5))])

col_transformer1 = ColumnTransformer([('cat_preproc', pipe_cat, ['Embarked', 'Sex'])],
                                     remainder='passthrough',
                                     force_int_remainder_cols=False)

model = DecisionTreeClassifier(random_state=0)

final_pipe = Pipeline([('preproc1', col_transformer1), ('num_inputer', pipe_num), ('model', model)])

In [21]:

final_pipe

Pipeline(steps=[('preproc1',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('cat_preproc',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Embarked', 'Sex'])])),
                ('num_inputer', Pipeline(steps=[('knn', KNNImputer())])),
                ('model', DecisionTreeClassifier(random_state=0))])

In [22]:
final_pipe.fit(X_train, y_train)

Pipeline(steps=[('preproc1',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('cat_preproc',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Embarked', 'Sex'])])),
                ('num_inputer', Pipeline(steps=[('knn', KNNImputer())])),
                ('model', DecisionTreeClassifier(random_state=0))])

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
accuracy_score(y_train, final_pipe.predict(X_train))

0.9798657718120806

In [25]:
accuracy_score(y_test, final_pipe.predict(X_test))

0.7559322033898305

### Метрики обучения

> Скор на трейне : 0.979
> Скор на валиде : 0.76
> Вывод достигли переобучение

### Исследование pipeline

In [26]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,male,54.0,0,0,51.8625,S
718,3,male,NaN,0,0,15.5000,Q


In [27]:
final_pipe.steps

[('preproc1',
  ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                    transformers=[('cat_preproc',
                                   Pipeline(steps=[('encoder',
                                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                   unknown_value=nan))]),
                                   ['Embarked', 'Sex'])])),
 ('num_inputer', Pipeline(steps=[('knn', KNNImputer())])),
 ('model', DecisionTreeClassifier(random_state=0))]

In [28]:
final_pipe.named_steps['preproc1'].named_transformers_['cat_preproc'].named_steps['encoder'].categories_

[array(['C', 'Q', 'S', nan], dtype=object),
 array(['female', 'male'], dtype=object)]

In [36]:
final_pipe.named_steps['model'].feature_importances_

array([0.04454388, 0.29283535, 0.10215548, 0.26618493, 0.05663961,
       0.03311038, 0.20453037])

In [37]:
X_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

Максимальное влияние параметров:
1. Sex - 0.29
2. SibSp (наличие родственников) - 0.27
3. Embarked (порт назначения) - 0.20


### Кроссвалидация

In [29]:

from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(final_pipe, X_train, y_train, cv=5,
                             scoring='accuracy')

In [30]:
cv_results

array([0.75      , 0.7394958 , 0.73109244, 0.74789916, 0.77310924])

## Предсказание на реальных данных

In [31]:
df_ground = pd.read_csv('data/test.csv')
df_ground.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [32]:
df_submission = pd.read_csv('data/gender_submission.csv')
df_submission.head(2)

,PassengerId,Survived
0,892,0
1,893,1


In [33]:
df_ground['Survived'] = final_pipe.predict(df_ground.drop('PassengerId', axis=1))


In [34]:
df_ground

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [35]:
df_ground[['PassengerId', 'Survived']].to_csv('data/test1.csv', index=False)

### Метрика на лидерборде

Score: 0.72966

place: 12335